### Association rules by Top 5 popular Departments 
#### Department - Frozen

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [6]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [ ]:
combine_train_prior

In [8]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="frozen"].reset_index()

In [9]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [10]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [11]:
grouped_data

,order_id,product_name
17,1145,Spinach Pizza
18,1145,Mexican Casserole Bowl
19,1145,Hash Brown Potato Patties
20,1145,Nacho Cheese & Bean Snacks
21,1145,"Eggs, Cheese & Turkey Sausage Breakfast Burrito"
22,1145,Original French Toast Sticks
23,1145,Strawberry Frozen Greek Yogurt Bars
30,1955,Traditional Bavarian Classic Soft Pretzels
31,1955,Light in Sodium Black Bean Vegetable Enchilada...
32,1955,Enchilada Black Bean Vegetable


In [12]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [13]:
#relace NAN with 0
table1=table1.fillna(0)

In [14]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [15]:
basket_sets = table1.applymap(encode_units)

In [21]:
## Running Apriori to get frequent itemset with a minimum support of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.009, use_colnames=True)

In [22]:
frequent_itemsets

,support,itemsets
0,0.014581,[Air Chilled Breaded Chicken Breast Nuggets]
1,0.013146,[Americone Dream® Ice Cream]
2,0.009804,[Bean & Cheese Burrito]
3,0.024423,[Berry Medley]
4,0.011346,[Birthday Cake Light Ice Cream]
5,0.010354,[Black Bean Tamale Verde]
6,0.009934,[Black Bean Vegetables Burrito]
7,0.058071,[Blueberries]
8,0.026171,[Broccoli & Cheddar Bake Meal Bowl]
9,0.011285,[Cafe Cuisine Sesame Chicken]


In [23]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [24]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Sliced Peaches),(Frozen Organic Wild Blueberries),0.027430,0.051708,0.009194,0.335188,6.482325,0.007776,1.426406
1,(Frozen Organic Wild Blueberries),(Organic Sliced Peaches),0.051708,0.027430,0.009194,0.177807,6.482325,0.007776,1.182898
2,(Organic Whole Strawberries),(Frozen Organic Wild Blueberries),0.062336,0.051708,0.023164,0.371603,7.186581,0.019941,1.509066
3,(Frozen Organic Wild Blueberries),(Organic Whole Strawberries),0.051708,0.062336,0.023164,0.447986,7.186581,0.019941,1.698622
4,(Organic Sliced Peaches),(Organic Whole Strawberries),0.027430,0.062336,0.011597,0.422809,6.782702,0.009888,1.624530
5,(Organic Whole Strawberries),(Organic Sliced Peaches),0.062336,0.027430,0.011597,0.186047,6.782702,0.009888,1.194872
6,(Organic Frozen Mango Chunks),(Organic Whole Strawberries),0.025843,0.062336,0.010499,0.406259,6.517204,0.008888,1.579247
7,(Organic Whole Strawberries),(Organic Frozen Mango Chunks),0.062336,0.025843,0.010499,0.168421,6.517204,0.008888,1.171455
8,(Organic Frozen Mango Chunks),(Frozen Organic Wild Blueberries),0.025843,0.051708,0.009057,0.350458,6.777634,0.007720,1.459939
9,(Frozen Organic Wild Blueberries),(Organic Frozen Mango Chunks),0.051708,0.025843,0.009057,0.175151,6.777634,0.007720,1.181013


In [25]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Sliced Peaches),(Frozen Organic Wild Blueberries),0.027430,0.051708,0.009194,0.335188,6.482325,0.007776,1.426406
1,(Frozen Organic Wild Blueberries),(Organic Sliced Peaches),0.051708,0.027430,0.009194,0.177807,6.482325,0.007776,1.182898
2,(Organic Whole Strawberries),(Frozen Organic Wild Blueberries),0.062336,0.051708,0.023164,0.371603,7.186581,0.019941,1.509066
3,(Frozen Organic Wild Blueberries),(Organic Whole Strawberries),0.051708,0.062336,0.023164,0.447986,7.186581,0.019941,1.698622
4,(Organic Sliced Peaches),(Organic Whole Strawberries),0.027430,0.062336,0.011597,0.422809,6.782702,0.009888,1.624530
5,(Organic Whole Strawberries),(Organic Sliced Peaches),0.062336,0.027430,0.011597,0.186047,6.782702,0.009888,1.194872
6,(Organic Frozen Mango Chunks),(Organic Whole Strawberries),0.025843,0.062336,0.010499,0.406259,6.517204,0.008888,1.579247
7,(Organic Whole Strawberries),(Organic Frozen Mango Chunks),0.062336,0.025843,0.010499,0.168421,6.517204,0.008888,1.171455
8,(Organic Frozen Mango Chunks),(Frozen Organic Wild Blueberries),0.025843,0.051708,0.009057,0.350458,6.777634,0.007720,1.459939
9,(Frozen Organic Wild Blueberries),(Organic Frozen Mango Chunks),0.051708,0.025843,0.009057,0.175151,6.777634,0.007720,1.181013


In [26]:
final_result.to_csv("Department_FrozenApriori.csv",sep=',')

In [27]:
test1=pd.read_csv("Department_FrozenApriori.csv")

In [28]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [29]:
test1=test1.replace({"u'": ''}, regex=True)

In [30]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [31]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [32]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [33]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Organic Sliced Peaches,Frozen Organic Wild Blueberries,0.027430,0.051708,0.009194,0.335188,6.482325,0.007776,1.426406
1,Frozen Organic Wild Blueberries,Organic Sliced Peaches,0.051708,0.027430,0.009194,0.177807,6.482325,0.007776,1.182898
2,Organic Whole Strawberries,Frozen Organic Wild Blueberries,0.062336,0.051708,0.023164,0.371603,7.186581,0.019941,1.509066
3,Frozen Organic Wild Blueberries,Organic Whole Strawberries,0.051708,0.062336,0.023164,0.447986,7.186581,0.019941,1.698622
4,Organic Sliced Peaches,Organic Whole Strawberries,0.027430,0.062336,0.011597,0.422809,6.782702,0.009888,1.624530
5,Organic Whole Strawberries,Organic Sliced Peaches,0.062336,0.027430,0.011597,0.186047,6.782702,0.009888,1.194872
6,Organic Frozen Mango Chunks,Organic Whole Strawberries,0.025843,0.062336,0.010499,0.406259,6.517204,0.008888,1.579247
7,Organic Whole Strawberries,Organic Frozen Mango Chunks,0.062336,0.025843,0.010499,0.168421,6.517204,0.008888,1.171455
8,Organic Frozen Mango Chunks,Frozen Organic Wild Blueberries,0.025843,0.051708,0.009057,0.350458,6.777634,0.007720,1.459939
9,Frozen Organic Wild Blueberries,Organic Frozen Mango Chunks,0.051708,0.025843,0.009057,0.175151,6.777634,0.007720,1.181013


In [35]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_FrozenApriori.csv")

In [34]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Organic Sliced Peaches,Frozen Organic Wild Blueberries,0.027430,0.051708,0.009194,0.335188,6.482325,0.007776,1.426406
1,Frozen Organic Wild Blueberries,Organic Sliced Peaches,0.051708,0.027430,0.009194,0.177807,6.482325,0.007776,1.182898
2,Organic Whole Strawberries,Frozen Organic Wild Blueberries,0.062336,0.051708,0.023164,0.371603,7.186581,0.019941,1.509066
3,Frozen Organic Wild Blueberries,Organic Whole Strawberries,0.051708,0.062336,0.023164,0.447986,7.186581,0.019941,1.698622
4,Organic Sliced Peaches,Organic Whole Strawberries,0.027430,0.062336,0.011597,0.422809,6.782702,0.009888,1.624530
5,Organic Whole Strawberries,Organic Sliced Peaches,0.062336,0.027430,0.011597,0.186047,6.782702,0.009888,1.194872
6,Organic Frozen Mango Chunks,Organic Whole Strawberries,0.025843,0.062336,0.010499,0.406259,6.517204,0.008888,1.579247
7,Organic Whole Strawberries,Organic Frozen Mango Chunks,0.062336,0.025843,0.010499,0.168421,6.517204,0.008888,1.171455
8,Organic Frozen Mango Chunks,Frozen Organic Wild Blueberries,0.025843,0.051708,0.009057,0.350458,6.777634,0.007720,1.459939
9,Frozen Organic Wild Blueberries,Organic Frozen Mango Chunks,0.051708,0.025843,0.009057,0.175151,6.777634,0.007720,1.181013


In [196]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()